In [11]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime
import sqlite3    
import re   
import requests   
from bs4 import BeautifulSoup   
import plotly.graph_objects as go
#import win32com.client as win32
import plotly.io as pio
from plotly.subplots import make_subplots

In [12]:
pd.set_option('display.max_rows', None)

#### Funções Data

In [13]:
def day_n (offset):         
    
    return datetime.today()-timedelta(days = offset)         

In [14]:
def sunday(offset, day):    
    
    if day.weekday()==6:     
        day = day_n(offset+2)         
        if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
            return day         
        else:              return holiday(offset, day, dataf)         
    
    return day   

####  Caso seja domingo essa função retorna o última dia antes de começar o final de semana, seja ele feriado ou não.  ####  Caso não seja domingo ele retorna o mesmo dia que foi passado como argumento pela função, sem alterá-lo.   

In [15]:
def saturday(offset, day):
    
    if day.weekday()==5:     
        day = day_n(offset+1)         
        if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
            return day         
        else:              return holiday(offset, day, dataf)     
    return day   

####  Caso seja sabado essa função retorna o última dia antes de começar o final de semana, 
####  seja ele feriado ou não.  
####  Caso não seja sábado ele retorna o mesmo dia que foi passado como argumento pela função, sem alterá-lo.   

In [16]:
def holiday(offset, day, dataf):     
    
    for i in dataf:     
        if (i[8:]+"-"+i[5:7]+"-"+i[0:4])==day.strftime("%d-%m-%Y"):             
            return holiday(offset+1, day_n(offset+1), dataf)         
    
            
    if day.strftime("%d-%m-%Y") == sunday(offset, day).strftime("%d-%m-%Y"):         
    
        
        if day.strftime("%d-%m-%Y") == saturday(offset, day).strftime("%d-%m-%Y"):             
            
            return day         
        else:             
            return saturday(offset, day)         
    else:         
        return sunday(offset, day)         

In [17]:
##### Descobrindo o D-1 #####

def find_last_workday( offset, day, dataf): 
    
    if day.strftime("%d-%m-%Y") == holiday(offset, day, dataf).strftime("%d-%m-%Y"):         
        
        if day.strftime("%d-%m-%Y") == sunday(offset, day).strftime("%d-%m-%Y"):             
            
            if day.strftime("%d-%m-%Y") == saturday(offset, day).strftime("%d-%m-%Y"):                 
                return day                 
                
            else:                     
                return saturday(offset, day)         
        else:                 
            return sunday(offset, day)         
    else:         
        return holiday(offset, day, dataf)          

##### Vetor de feriados

In [18]:
def holidays():
    
    feriados = pd.read_csv(r"C:\Users\psilv\Python\feriados.csv")   
 
 
    data = feriados['data'].tolist()   
 
    ########### Cria vetor de feriados 
    
    dataf=[] 
    
    for i in data:   
        
        dataf.append(i.strip()[-4:] + "-" + ("0"+ i.strip()[0] if i.find("/")==1 else i.strip()[0:i.find("/")]) + "-" + ("0" + re.search("/(.*?)/", i.strip()).group(1) if len(re.search("/(.*?)/", i.strip()).group(1))==1 else re.search("/(.*?)/", i.strip()).group(1)))
        
    return dataf     

In [19]:
def past_n_days(n):     
    day = datetime.today()-timedelta(days = n)     
    print("O último dia corrido é: " +  day.strftime("%d-%m-%Y"))     
    return day   

#####  Função auxiliar para criar Vetor de dias úteis 

In [20]:
##### Date Format input  YY-MM-dd,  run_days são números inteiros

##### O input de data dentro da função pandas.date_range deve ser no formato YY-MM-dd

def date_array (last_day, run_days):

   
    Datas = pd.date_range(end = last_day, periods = run_days).to_pydatetime().tolist()

    
    datas=[] 

    
    for k in range(len(Datas)):

        datas.append(Datas[len(Datas) - k -1])



 
    return datas

#### Função iterativa para criar Vetor de dias úteis.

In [21]:
#### Função para obter um vetor de dias úteis de um tamanho específico, por ser iterativa é custosa computacionalmente

#### Date format input dd-MM-YY, length é número inteiro


def work_days_array_length(last_day, length):


    i=length

    while len(date_array(last_day, i))!= length:

        if len(date_array(last_day, i))==length:

            break

        i+=1

    return date_array(last_day, i)

In [22]:
####### Criando vetor desde o ultimo dia do banco até o D-1

####### Start é a mais antiga, e End a mais recente

####### Date Format input  YY-MM-dd (SQL)


def  work_days_array_btw_dates(start_date, end_date):
    

    end = date(int(end_date[:4]), int(end_date[5:7]), int(end_date[-2:]))

    start = date(int(start_date[:4]), int(start_date[5:7]), int(start_date[-2:]))

    
    dif = end-start
    

    dif = int(abs(dif.days))
    
    
    print("Números de dias a serem atualizados:  " + str(dif))
    print("\n")
    
    #### No caso start_date=end_date é preciso desse if 
    
    if dif == 0:
        dif=2
            


    dates = date_array(end_date, 2*dif)

    aux=0
    

    for i in dates:

        if i.strftime("%Y-%m-%d")==start_date:

            aux = dates.index(i)
                 
    
    return dates[:aux+1]

#### Função para deletar linha específica

In [23]:
def delete_row_sql(table_name, column_name, conn, desired_info):
    
    
    query = ("""DELETE""" + 
             """ FROM """ + table_name +
             """ WHERE """ + column_name +
             """ = '""" + desired_info +
             """'; """)
    
    conn.execute(query)
    
    print("Deleted as Follows:")
    print("\n")
    print(query)

#### Função para obter um dataframe a partir de uma query do banco de dados

In [24]:
### Criar função para obter um dataframe entre datas específicas a partir de uma query em SQL

### MELHORAR PRA RANGE DE DATAS


def df_from_sql_query(table_name, date_column_name, conn, start_date, end_date):
    
    
    query = ("""SELECT * """ + 
             """ FROM """ + table_name +
             """ WHERE """ + date_column_name +
             """ BETWEEN strftime('%Y-%m-%d %H:%M:%S', '""" + start_date +
             """') AND strftime('%Y-%m-%d %H:%M:%S', '""" + end_date +
             """') ORDER BY """ + date_column_name + """; """)
    
    #print(query)

    df = pd.read_sql_query(query, conn)
    
    
    return df   

In [25]:
# df_from_sql_query('bitcoin', "Date", conn, "2022-01-28", "2022-07-25")

#### Inserindo novas linhas no banco de dados

In [26]:
##### Tornar essa função genérica para qualquer dataframe

def insert_n_lines_database(cursor, df, table_name, date_column_name):
    
           
    print("\n")     
    print("Os dados antes da atualização eram:")   
    print("\n")  
    
    for i in get_last_n_rows(cursor, table_name, 5, date_column_name):   
        print(i)             
 
    print("\n")     
   
    insert_n_rows(cursor, df, table_name)             
       
    print("\n")   
    print("Os novos dados são:")   
    print("\n")     
                   
    for i in get_last_n_rows(cursor, table_name, 5, date_column_name):   
        print(i)         
    print("\n")             
            

#### Atualizando as bases de dados

In [27]:
def updating_database(cursor, date_column_name):      
    
    for i in assets:
        
        print('\n')
        
        day = past_n_days(1)
    
        day = day.strftime("%Y-%m-%d")
        
        ###### Verificando a última data atualizada no banco de dados   
        last_date_db = get_last_db_date(cursor, date_column_name, i)[:10]        

        if last_date_db == day:         
            print("Os dados da cripto " + str(i) + " estão Atualizados!!")         
        else:         
            print("\n")     
            print("O último dia atualizado no banco de dados é: " + last_date_db)   
            print("\n")         

            #### Vetor de datas   
            dates = work_days_array_btw_dates(last_date_db, day)[:-1]   
            print("Datas a serem atualizadas no banco de dados:")         
            print("\n")         

            for j in range(len(dates)):             
                print(dates[j])             


            print("\n")
            print("Downloadind from Yahoo Finance API ....")         
            print("\n")         

            #### Dataframe com os resultados
            
            print("Moeda Atualizada: " + assets[i])
    

            df = get_data_btw_date(assets[i],dates[-1:][0].strftime("%Y-%m-%d"), dates[0].strftime("%Y-%m-%d"))     


            #### Inserindo novos dados           
            insert_n_lines_database(cursor, df, i, date_column_name)  

            ##### Adicionar o insert lines dentro da função Web Scrap para atualizar o banco de dados em cada iteração e não
            ##### correr o risco de falhar o web scraping e perder os dados
    
    

#### Função para captar os dados da API do Yahoo Finance

In [28]:
#### Nome da moeda, exemplo: Ethereum é ETH-USD
#### Data format YY-MM-DD

def get_data_btw_date(crypto_name, start_date, end_date):
    
    crypto_df = yf.download(crypto_name, start= start_date, end= end_date)
    
    crypto_df = crypto_df.reset_index(level=0)
    
    
    crypto_df['Date'] = pd.to_datetime(crypto_df['Date'], format='%Y-%m-%d %H:%M:%S')
    
    crypto_df['Date'] = crypto_df['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    

    return crypto_df

In [29]:
get_data_btw_date('BTC-USD', '2022-08-21', '2022-08-21')

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-08-21 00:00:00,21160.392578,21668.845703,21103.197266,21534.121094,21534.121094,23102307723


#### Funções SQL

In [30]:
## Insert crypto name in lower case

In [31]:
#### index can be False or True, to me worked as False

def create_table(df, name_desired, conn, index):
    
        df.to_sql(name_desired, conn, if_exists='fail', index=index)   
    


In [32]:
def rename_table(table_name, new_table_name, cursor):
    
    try:
        
        for i in get_names_tables(cursor):
        
            if new_table_name == i[0]:
                
                print("O Nome já está em uso!")
            
                raise Exception

        cursor.execute("""ALTER TABLE """ +  table_name + """ RENAME TO """ + new_table_name + """ ;""")

    except Exception:

        print("Não foi possível alterar o nome da tabela!")
        

In [33]:
def show_table(table_name, cursor):     
    
    c=cursor.execute("""SELECT * FROM """ + table_name ).fetchall()     
    for i in c:      
        print(i)     
    return c     

In [34]:
def delete_table(name, cursor):   
    
    cursor.execute("""DROP TABLE """ +  name ).fetchall()   

In [35]:
def last_database_day(crypto_name):

    sql_str = '''SELECT Date FROM ''' + crypto_name + ''' ORDER BY Date DESC LIMIT 1;'''

    c.execute(sql_str)

    date = c.fetchone()

    return date[0][:10]
    

In [36]:
def get_names_tables(cursor):     
    
    cursor.execute('''SELECT name FROM sqlite_master WHERE type='table';''')   
    
    return(cursor.fetchall())

In [37]:
def count_rows_table(cursor, table_name): 
    
    c = cursor.execute("""SELECT COUNT(*) FROM """ + table_name).fetchall()     
    return c[0][0]   

In [38]:
def get_columns_names(cursor, table_name): 
    
    c = cursor.execute("""PRAGMA table_info(""" + table_name + """);  """ ).fetchall()     
    col_names = []     
    for i in c:         
            col_names.append(i[1])
            
    return col_names   

In [39]:
def get_last_n_rows(cursor, table_name, n_rows, column_name):
    
    c = cursor.execute("""SELECT * FROM """ 
                       + table_name + 
                       """ ORDER BY """ +column_name + 
                       """ DESC LIMIT """+ str(n_rows) + 
                       """ ; """  ).fetchall()     
#     for i in c:      
#         print(i)


    
    return c     

In [40]:
def insert_n_rows(cursor, df, table_name):     
 
   
    new_values = []   
    
    for i in range(len(df)):    
        
        new_values.append(tuple(df.iloc[i]))         
    
    s=""     
    
    for i in new_values:   
        
        s+=str(i) + ","   
        
    
    s=s[:-1] 

    #print(s)
    
    
    cursor.execute("""INSERT INTO """ + table_name + """ VALUES """ + s + """ ; """  )     
    

In [41]:
def distinct_values(table_name, column_name, cursor):
    
    c = cursor.execute("""SELECT """ + column_name + """, COUNT(*) """ + """ FROM """ + table_name + """ GROUP BY """ +  column_name + """ ; """  ).fetchall()     
    
    
    for i in c:
        
        print(i)
    
       

In [42]:
def change_columns_names(cursor, table_name, column_old_name, column_new_name):   
 
    print("Old Columns Names: " + str(get_columns_names(cursor, table_name)) )     
    c = cursor.execute("""ALTER TABLE """ + table_name +                                 """ RENAME COLUMN """ + column_old_name +                                 """ TO """+column_new_name+                                 """ ; """  ).fetchall()     
    print("New Columns Names: " + str(get_columns_names(cursor, table_name)) )     
 

In [43]:
def get_last_db_date(cursor,column_name, table_name):  
    
    c=cursor.execute("""SELECT """ + column_name + 
                     """ FROM """ + table_name + 
                     """ ORDER BY """ + column_name + 
                     """ DESC LIMIT 1 ; """).fetchall()     
    return c[0][0]   

In [44]:
def database_info(cursor, table_name, date_column_name):
    
        
    
    print("O banco de dados contém as seguintes tabelas: ")
    
    print("\n")
    
    for i in get_names_tables(cursor):
        print(i[0])
    
    print("\n")
    
    print("O nome da tabela solicitada é :" + table_name)
    
    print("\n")
    
    print("O nome das colunas são: ")
    
    print("\n")
    
    print(get_columns_names(cursor, table_name))
    
    print("\n")
    
    print("O número total de linhas é: " + str(count_rows_table(cursor, table_name)))
    
    print("\n")
    
    print("A data mais recente dos dados é: " + str(get_last_db_date(cursor,date_column_name, table_name)))
    
    print("\n")
    
    print("Todas as datas distintas e quantas linhas cada uma possui são: ")
    
    print("\n")
    
    print(distinct_values(table_name, date_column_name, cursor)) 
    
    

In [45]:
def get_new_crypto_in_yf(crypto_name, table_title, conn, end_date, index=False):
    
 
    df = get_data_btw_date(crypto_name, '2000-01-01', end_date)
        
    #### Data escolhida é bem antes do surgimento das criptomoedas no planeta terra
        
        
    create_table(df, table_title, conn, index)
    

In [46]:
###### updating entire database for all assets ##########

def insert_new_criptos_in_db(assets):
    
    day = past_n_days(1)
    
    day = day.strftime("%Y-%m-%d")

    for i in assets:
        
        try:

            get_new_crypto_in_yf(assets[i], i, conn, day, index=False)
            
        
        except Exception:
            
            print(str(assets[i]+ " já  está no banco de dados!"))
            
            continue

In [47]:
assets = { 'BTC': 'BTC-USD', 
           'ETH': 'ETH-USD', 
           'USDT':'USDT-USD', 
           'USDC':'USDC-USD', 
           'BNB':'BNB-USD',
           'BUSD':'BUSD-USD',
           'XRP': 'XRP-USD',
           'ADA': 'ADA-USD',
           'SOL': 'SOL-USD',
           'DOGE':'DOGE-USD',
           'DOT': 'DOT-USD',
           'HEX': 'HEX-USD',
           'SHIB': 'SHIB-USD',
           'DAI': 'DAI-USD',
           'WTRX': 'WTRX-USD',
           'AVAX':'AVAX-USD',
           'MATIC': 'MATIC-USD',
           'TRX': 'TRX-USD',
           'STETH': 'STETH-USD',
           'WBTC': 'WBTC-USD',
           'UNI1': 'UNI1-USD',
           'LEO': 'LEO-USD',
           'ETC': 'ETC-USD',
           'LTC': 'LTC-USD'
         }

#### Rodando

In [48]:
conn = sqlite3.connect('crypto_data')
c = conn.cursor()

In [49]:
updating_database(c, "Date")



O último dia corrido é: 14-09-2022


O último dia atualizado no banco de dados é: 2022-08-19


Números de dias a serem atualizados:  26


Datas a serem atualizadas no banco de dados:


2022-09-14 00:00:00
2022-09-13 00:00:00
2022-09-12 00:00:00
2022-09-11 00:00:00
2022-09-10 00:00:00
2022-09-09 00:00:00
2022-09-08 00:00:00
2022-09-07 00:00:00
2022-09-06 00:00:00
2022-09-05 00:00:00
2022-09-04 00:00:00
2022-09-03 00:00:00
2022-09-02 00:00:00
2022-09-01 00:00:00
2022-08-31 00:00:00
2022-08-30 00:00:00
2022-08-29 00:00:00
2022-08-28 00:00:00
2022-08-27 00:00:00
2022-08-26 00:00:00
2022-08-25 00:00:00
2022-08-24 00:00:00
2022-08-23 00:00:00
2022-08-22 00:00:00
2022-08-21 00:00:00
2022-08-20 00:00:00


Downloadind from Yahoo Finance API ....


Moeda Atualizada: BTC-USD
[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


('2022-08-19 00:00:00', 23213.3125, 23213.3125, 20868.84765625, 20877.552734375, 20877.552734375, 40509610260)
('2

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


('2022-08-19 00:00:00', 298.15277099609375, 298.1907043457031, 278.5794372558594, 279.7187194824219, 279.7187194824219, 1598319492)
('2022-08-18 00:00:00', 306.7964782714844, 310.1879577636719, 297.02642822265625, 298.1412353515625, 298.1412353515625, 1041814513)
('2022-08-17 00:00:00', 316.401611328125, 323.2359924316406, 305.33489990234375, 306.63641357421875, 306.63641357421875, 1111784114)
('2022-08-16 00:00:00', 319.7127685546875, 322.1582946777344, 312.8060607910156, 316.4236755371094, 316.4236755371094, 963072351)
('2022-08-15 00:00:00', 318.032470703125, 326.8336486816406, 314.85809326171875, 319.8872985839844, 319.8872985839844, 1141553086)




Os novos dados são:


('2022-09-14 00:00:00', 277.4132995605469, 282.1502685546875, 274.83056640625, 279.3858642578125, 279.3858642578125, 956413768)
('2022-09-13 00:00:00', 293.64923095703125, 297.6877136230469, 275.968963623046

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


('2022-08-19 00:00:00', 39.54132843017578, 39.54132843017578, 35.755027770996094, 35.8902587890625, 35.8902587890625, 1821467945)
('2022-08-18 00:00:00', 40.426509857177734, 41.607303619384766, 39.525936126708984, 39.545230865478516, 39.545230865478516, 885648540)
('2022-08-17 00:00:00', 43.15305709838867, 44.633087158203125, 40.28676986694336, 40.403053283691406, 40.403053283691406, 1336772691)
('2022-08-16 00:00:00', 43.806095123291016, 44.1871223449707, 42.78215408325195, 43.15880584716797, 43.15880584716797, 1131452045)
('2022-08-15 00:00:00', 44.92243957519531, 47.127540588378906, 42.90903091430664, 43.82322311401367, 43.82322311401367, 1364253429)




Os novos dados são:


('2022-09-14 00:00:00', 33.01827621459961, 34.28319549560547, 32.870452880859375, 34.09157180786133, 34.09157180786133, 1003604224)
('2022-09-13 00:00:00', 37.421607971191406, 38.870384216308594, 32.8704

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


('2022-08-19 00:00:00', 1.4000000192027073e-05, 1.4000000192027073e-05, 1.2999999853491317e-05, 1.2999999853491317e-05, 1.2999999853491317e-05, 1128117927)
('2022-08-18 00:00:00', 1.4999999621068127e-05, 1.4999999621068127e-05, 1.4000000192027073e-05, 1.4000000192027073e-05, 1.4000000192027073e-05, 557409408)
('2022-08-17 00:00:00', 1.5999999959603883e-05, 1.5999999959603883e-05, 1.4999999621068127e-05, 1.4999999621068127e-05, 1.4999999621068127e-05, 1047374797)
('2022-08-16 00:00:00', 1.4999999621068127e-05, 1.5999999959603883e-05, 1.4999999621068127e-05, 1.5999999959603883e-05, 1.5999999959603883e-05, 1302891507)
('2022-08-15 00:00:00', 1.700000029813964e-05, 1.700000029813964e-05, 1.4999999621068127e-05, 1.4999999621068127e-05, 1.4999999621068127e-05, 2327359499)




Os novos dados são:


('2022-09-14 00:00:00', 1.2000000424450263e-05, 1.2000000424450263e-05, 1.20000004244502

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


('2022-08-19 00:00:00', 0.859561026096344, 0.859561026096344, 0.7677389979362488, 0.7721350193023682, 0.7721350193023682, 733235258)
('2022-08-18 00:00:00', 0.8833839893341064, 0.9076340198516846, 0.8596519827842712, 0.8596519827842712, 0.8596519827842712, 377740073)
('2022-08-17 00:00:00', 0.9357380270957947, 0.9629669785499573, 0.8799059987068176, 0.8831790089607239, 0.8831790089607239, 474243132)
('2022-08-16 00:00:00', 0.9517729878425598, 0.9650070071220398, 0.9286059737205505, 0.9357960224151611, 0.9357960224151611, 404175969)
('2022-08-15 00:00:00', 1.000846028327942, 1.0317469835281372, 0.9352419972419739, 0.9520350098609924, 0.9520350098609924, 560505268)




Os novos dados são:


('2022-09-14 00:00:00', 0.8462330102920532, 0.8672659993171692, 0.8377940058708191, 0.8647890090942383, 0.8647890090942383, 414726283)
('2022-09-13 00:00:00', 0.9300410151481628, 0.940589010715

[*********************100%***********************]  1 of 1 completed


Os dados antes da atualização eram:


('2022-08-19 00:00:00', 7.600599765777588, 7.657313823699951, 6.8590497970581055, 6.909179210662842, 6.909179210662842, 232362260)
('2022-08-18 00:00:00', 7.941886901855469, 8.183577537536621, 7.588263988494873, 7.601743221282959, 7.601743221282959, 157458896)
('2022-08-17 00:00:00', 8.343829154968262, 8.645283699035645, 7.901493072509766, 7.939789772033691, 7.939789772033691, 172065284)
('2022-08-16 00:00:00', 8.477188110351562, 8.548687934875488, 8.222295761108398, 8.346027374267578, 8.346027374267578, 176881933)
('2022-08-15 00:00:00', 8.883331298828125, 9.134861946105957, 8.34613037109375, 8.479776382446289, 8.479776382446289, 201649381)




Os novos dados são:


('2022-09-14 00:00:00', 6.044609069824219, 6.22540283203125, 5.998297214508057, 6.188218116760254, 6.188218116760254, 100209429)
('2022-09-13 00:00:00', 6.651512145996094, 6.7234601974487305, 6.024077892303467, 6.11

In [50]:
# database_info(c, 'ETH','Date')

In [51]:
def plot(title,crypto, conn, startDate, endDate):
    
    
    df=df_from_sql_query(crypto, "Date", conn, startDate, endDate)
    
    df['Date']= pd.to_datetime(df['Date'])
    
    df=df.drop(columns=['Adj Close'])
    
#     eth_df.set_index('Date')

    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False)

    fig.update_layout(
        title= title,
        yaxis_title='Price',

    )

    fig.write_html(title + ".html")
    fig.write_html(title + ".html")

    fig.show()

In [55]:
plot('ETHER-USD','ETH', conn, '2022-08-01', '2022-09-14')

In [56]:
plot('BTC-USD','BTC', conn, '2022-08-01', '2022-09-14')

In [54]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'paasxx@gmail.com'
mail.Subject = 'Criptomoedas - Destaques'
mail.Body = ''

with open('test.html', 'r') as myfile:
    data=myfile.read()
mail.HTMLBody = "Please Find the Report here " + data


# To attach a file to the email (optional):
signatureimage = "C:\\Users\\psilv\\Python\\test.html"
mail.Attachments.Add(signatureimage)

# mail.HTMLBody = "<html><body><img src=""cid:test.html""></body></html>"


# mail.Send()
mail.Display()

NameError: name 'win32' is not defined